In [1]:
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_recall_curve, auc, confusion_matrix, classification_report, recall_score, roc_auc_score, precision_score 
import dalex as dx

from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
from fairtorch import ConstraintLoss, DemographicParityLoss, EqualiedOddsLoss
import random
import numpy as np
import torch
import os
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [2]:
netherlands = pd.read_excel('../data/42256_2020_253_MOESM1_ESM.xlsx')

df = netherlands[["Age", "Gender", "LD", "CRP", "Percentage lymphocytes", "Survival/death"]]
df = df.assign(death = np.where(df["Survival/death"] == 'Alive', 0, 1))
df = df[["Age", "Gender", "LD", "CRP", "Percentage lymphocytes", "death"]]
X = df.drop("death", axis = 1)
y_netherlands = df.death

netherlands = df
y_netherlands = df.death
netherlands = netherlands.iloc[:,2:-1]
netherlands['death'] = y_netherlands
netherlands['from'] = 'netherlands'
netherlands = netherlands.rename(columns = { 'Percentage lymphocytes':'Lymphocytes', 'LD':'LDH'})

In [3]:
import sys
sys.path.append("..")
china = pd.read_excel('../data/time_series_375_prerpocess_en.xlsx')

from raport_v1.utils_features_selection import data_read_and_split
from raport_v1.utils_features_selection import data_read_and_split
X_data_all_features, Y_data, x_col = data_read_and_split()
cheng = {'乳酸脱氢酶':'LDH', '超敏C反应蛋白':'CRP', '淋巴细胞(%)':'Lymphocytes'}
blood_df = X_data_all_features[list(cheng.keys())]
blood_df.columns = list(cheng.values())
china = blood_df
china['death'] = Y_data
china['from'] = 'china'
china = china.reset_index(drop = True)

In [4]:
ny = pd.read_csv('~/Downloads/Yan_reply_First_last_wtime.csv')

y_ny = ny.Expired_Outcome
newyork = ny[['First_LDH', 'First_CRP', 'First_Lymph']]
newyork = newyork.rename(columns = {'First_LDH':'LDH',
                                      'First_CRP':'CRP',
                                      'First_Lymph':'Lymphocytes'})
newyork['from'] = 'newyork'
newyork['death'] = y_ny

In [5]:
nations = china.append([newyork, netherlands])
nations['death'] = nations.death.astype('int')
nations = nations.reset_index(drop=True)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
X = nations.drop('death', axis =1)
y = nations.death

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

A_train = np.where(X_train['from']=='china', 'china', 'other')
X_train = X_train.drop('from', axis = 1)


A_test = np.where(X_test['from']=='china', 'china', 'other')
X_test = X_test.drop('from', axis = 1)


In [9]:
std = StandardScaler()

In [10]:
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

# Mitigation

In [11]:
# ! pip install fairtorch
# ! pip install fairlearn

In [12]:
from fairlearn.reductions import EqualizedOdds
from fairlearn.reductions import ExponentiatedGradient
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

In [44]:
clf = HistGradientBoostingClassifier(monotonic_cst=[1, -1, -1])
clf.fit(X_train_std, y_train)

HistGradientBoostingClassifier(monotonic_cst=[1, -1, -1])

In [45]:
exp = dx.Explainer(clf, X_test_std, y_test)

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 430 rows 3 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 430 values
  -> model_class       : sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7fae5ff32e18> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00208, mean = 0.329, max = 0.965
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.824, mean = -0.0155, max = 0.966
  -> model_info        : package sklearn

A

In [46]:
exp.model_performance().result

,recall,precision,f1,accuracy,auc
HistGradientBoostingClassifier,0.451852,0.54955,0.495935,0.711628,0.757514


In [47]:
exp.model_profile().plot()

Calculating ceteris paribus: 100%|██████████| 3/3 [00:00<00:00,  9.99it/s]


In [17]:
fo = exp.model_fairness(protected=A_test, privileged = "china")
fo.plot()


In [18]:
reductions = ExponentiatedGradient(clf, EqualizedOdds(ratio_bound=0.98))
reductions.fit(X_train_std, y_train, sensitive_features = A_train)

In [19]:
exp2 = dx.Explainer(reductions,
                    X_test_std,
                    y_test,
                    label = 'reduced',
                    model_type = 'classification', verbose = False)
fo2 = exp2.model_fairness(protected=A_test, privileged = "china")
fo2.plot(fo)

In [20]:
fo2.plot()

In [21]:
fo2.plot(fo,
         type='performance_and_fairness',
         performance_metric = 'precision',
         fairness_metric = 'PPV')

In [22]:
exp2.model_performance()

,recall,precision,f1,accuracy,auc
reduced,0.37037,0.531915,0.436681,0.7,0.592116


Redukcja zadziałała, niestety nie na wszystkie metryki, ale na tą na której najbardziej nam zależy tak. Zmniejszyła niestety znacząco precyzję modelu

# Fairtorch

In [23]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(123)

X_tr = torch.from_numpy(X_train_std).float()
X_te = torch.from_numpy(X_test_std).float()

y_tr = torch.from_numpy(y_train.to_numpy()).float()
y_te = torch.from_numpy(y_test.to_numpy()).float()

model_before = nn.Sequential(nn.Linear(X_tr.shape[1],128), 
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 1),)


criterion = nn.BCEWithLogitsLoss(pos_weight=None)
optimizer = optim.Adam(model_before.parameters())


for param in model_before.parameters():
    nn.init.normal_(param, 0, 1e-3)
    
model_before.train()

for i in range(0, 400):
    optimizer.zero_grad()    
    logit = model_before(X_tr)
    loss = criterion(logit.view(-1), y_tr)
    loss.backward()
    optimizer.step()

acc_train = ((torch.sigmoid(model_before(X_tr)).view(-1) > 0.5 ).float()  == y_tr ).float().mean().float().item()
acc_test = ((torch.sigmoid(model_before(X_te)).view(-1) > 0.5 ).float()  == y_te ).float().mean().float().item()

print("acc train: ", acc_train)
print("acc test: ",acc_test)


acc train:  0.8027950525283813
acc test:  0.6883720755577087


In [24]:
def predict_torch(model, data):
    data = np.array(data)
    data_tensor = torch.from_numpy(data).float()
    return torch.sigmoid(model(data_tensor)).view(-1).detach().numpy()

In [25]:
exp_net = dx.Explainer(model_before,
                       X_test_std,
                       np.array(y_test),
                       predict_function=predict_torch,
                       model_type='classification', 
                       label = 'torch_before')

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 430 rows 3 cols
  -> target variable   : 430 values
  -> model_class       : torch.nn.modules.container.Sequential (default)
  -> label             : torch_before
  -> predict function  : <function predict_torch at 0x7fae01c7d2f0> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000601, mean = 0.364, max = 0.983
  -> model type        : classification will be used
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.948, mean = -0.0496, max = 0.995
  -> model_info        : package torch

A new explainer has been created!


In [26]:
exp_net.model_performance().result

,recall,precision,f1,accuracy,auc
torch_before,0.540741,0.503448,0.521429,0.688372,0.6914


In [27]:
fobject = exp_net.model_fairness(protected=A_test, privileged = "china")
fobject.plot()

In [28]:
from sklearn.preprocessing import OrdinalEncoder

In [29]:
oe = OrdinalEncoder()
A_train_ord = oe.fit_transform(np.array(A_train).reshape(-1,1))
A_train_ord = A_train_ord.reshape(-1)

A_test_ord = oe.transform(np.array(A_test).reshape(-1,1))
A_test_ord = A_test_ord.reshape(-1)


In [30]:
sensi_train = torch.from_numpy(A_train_ord.astype(int))
sensi_test = torch.from_numpy(A_test_ord.astype(int))

In [31]:
y_tr = y_tr.to('cpu')

In [32]:
torch.device('cpu')

device(type='cpu')

In [33]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(2020)

In [34]:

model_after = nn.Sequential(nn.Linear(X_tr.shape[1],128), 
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 1),)

dp_loss = EqualiedOddsLoss(sensitive_classes=[0, 1], alpha=50, y = y_tr)
optimizer = optim.Adam(model_after.parameters())



for param in model_after.parameters():
    nn.init.normal_(param, 0, 1e-2)
    model_after.train()

for i in range(0, 500):
    optimizer.zero_grad()    
    logit = model_after(X_tr)
    loss = criterion(logit.view(-1), y_tr)
    print(dp_loss(X_tr, logit, sensi_train))
    loss +=  dp_loss(X_tr, logit, sensi_train)
    loss.backward()
    optimizer.step()


acc_train = ((torch.sigmoid(model_after(X_tr)).view(-1) > 0.5 ).float()  == y_tr ).float().mean().float().item()
acc_test = ((torch.sigmoid(model_after(X_te)).view(-1) > 0.5 ).float()  == y_te ).float().mean().float().item()

print("acc train: ", acc_train)
print("acc test: ",acc_test)
acc_test_dp = acc_test

tensor(2.2972e-09, grad_fn=<MulBackward0>)
tensor(6.6603e-07, grad_fn=<MulBackward0>)
tensor(2.8319e-06, grad_fn=<MulBackward0>)
tensor(7.4559e-06, grad_fn=<MulBackward0>)
tensor(1.5795e-05, grad_fn=<MulBackward0>)
tensor(2.8428e-05, grad_fn=<MulBackward0>)
tensor(4.4547e-05, grad_fn=<MulBackward0>)
tensor(6.0713e-05, grad_fn=<MulBackward0>)
tensor(7.1892e-05, grad_fn=<MulBackward0>)
tensor(7.6311e-05, grad_fn=<MulBackward0>)
tensor(7.4256e-05, grad_fn=<MulBackward0>)
tensor(6.6468e-05, grad_fn=<MulBackward0>)
tensor(5.5844e-05, grad_fn=<MulBackward0>)
tensor(4.7602e-05, grad_fn=<MulBackward0>)
tensor(4.6358e-05, grad_fn=<MulBackward0>)
tensor(5.3978e-05, grad_fn=<MulBackward0>)
tensor(7.5650e-05, grad_fn=<MulBackward0>)
tensor(0.0001, grad_fn=<MulBackward0>)
tensor(0.0001, grad_fn=<MulBackward0>)
tensor(0.0001, grad_fn=<MulBackward0>)
tensor(9.5683e-05, grad_fn=<MulBackward0>)
tensor(5.7125e-05, grad_fn=<MulBackward0>)
tensor(3.4695e-05, grad_fn=<MulBackward0>)
tensor(3.3428e-05, grad

tensor(0.0128, grad_fn=<MulBackward0>)
tensor(0.0125, grad_fn=<MulBackward0>)
tensor(0.0133, grad_fn=<MulBackward0>)
tensor(0.0121, grad_fn=<MulBackward0>)
tensor(0.0124, grad_fn=<MulBackward0>)
tensor(0.0115, grad_fn=<MulBackward0>)
tensor(0.0117, grad_fn=<MulBackward0>)
tensor(0.0111, grad_fn=<MulBackward0>)
tensor(0.0116, grad_fn=<MulBackward0>)
tensor(0.0114, grad_fn=<MulBackward0>)
tensor(0.0140, grad_fn=<MulBackward0>)
tensor(0.0202, grad_fn=<MulBackward0>)
tensor(0.0499, grad_fn=<MulBackward0>)
tensor(0.0672, grad_fn=<MulBackward0>)
tensor(0.0240, grad_fn=<MulBackward0>)
tensor(0.0336, grad_fn=<MulBackward0>)
tensor(0.0204, grad_fn=<MulBackward0>)
tensor(0.0259, grad_fn=<MulBackward0>)
tensor(0.0166, grad_fn=<MulBackward0>)
tensor(0.0302, grad_fn=<MulBackward0>)
tensor(0.0150, grad_fn=<MulBackward0>)
tensor(0.0212, grad_fn=<MulBackward0>)
tensor(0.0186, grad_fn=<MulBackward0>)
tensor(0.0163, grad_fn=<MulBackward0>)
tensor(0.0236, grad_fn=<MulBackward0>)
tensor(0.0160, grad_fn=<M

tensor(0.0143, grad_fn=<MulBackward0>)
tensor(0.0144, grad_fn=<MulBackward0>)
tensor(0.0137, grad_fn=<MulBackward0>)
tensor(0.0140, grad_fn=<MulBackward0>)
tensor(0.0138, grad_fn=<MulBackward0>)
tensor(0.0132, grad_fn=<MulBackward0>)
tensor(0.0132, grad_fn=<MulBackward0>)
tensor(0.0133, grad_fn=<MulBackward0>)
tensor(0.0134, grad_fn=<MulBackward0>)
tensor(0.0129, grad_fn=<MulBackward0>)
tensor(0.0129, grad_fn=<MulBackward0>)
tensor(0.0131, grad_fn=<MulBackward0>)
tensor(0.0129, grad_fn=<MulBackward0>)
tensor(0.0126, grad_fn=<MulBackward0>)
tensor(0.0124, grad_fn=<MulBackward0>)
tensor(0.0128, grad_fn=<MulBackward0>)
tensor(0.0122, grad_fn=<MulBackward0>)
tensor(0.0121, grad_fn=<MulBackward0>)
tensor(0.0120, grad_fn=<MulBackward0>)
tensor(0.0121, grad_fn=<MulBackward0>)
tensor(0.0117, grad_fn=<MulBackward0>)
tensor(0.0115, grad_fn=<MulBackward0>)
tensor(0.0118, grad_fn=<MulBackward0>)
tensor(0.0114, grad_fn=<MulBackward0>)
tensor(0.0114, grad_fn=<MulBackward0>)
tensor(0.0117, grad_fn=<M

In [35]:
exp_net2 = dx.Explainer(model_after,
                        X_test_std,
                        np.array(y_test),
                        predict_function=predict_torch,
                        model_type='classification', 
                        label = 'torch_after')

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 430 rows 3 cols
  -> target variable   : 430 values
  -> model_class       : torch.nn.modules.container.Sequential (default)
  -> label             : torch_after
  -> predict function  : <function predict_torch at 0x7fae01c7d2f0> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 5.5e-05, mean = 0.35, max = 0.985
  -> model type        : classification will be used
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.859, mean = -0.0364, max = 0.937
  -> model_info        : package torch

A new explainer has been created!


In [36]:
exp_net2.model_performance().result

,recall,precision,f1,accuracy,auc
torch_after,0.525926,0.559055,0.541985,0.72093,0.737954


In [37]:
fobject2 = exp_net2.model_fairness(protected=A_test, privileged = "china")
fobject2.plot(fobject)

In [38]:
fobject.plot(fobject2, type = 'stacked')

In [39]:
fobject.result

,TPR,TNR,PPV,NPV,FNR,FPR,FDR,FOR,ACC,STP
china,1.000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,1.000000
other,0.808,0.751306,0.439525,1.061662,1.32,6.534884,8.013514,0.818898,0.822943,1.092357
